In [ ]:
!pip3 install torch==1.13.0 torchvision torchaudio torchtext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

torch.manual_seed(0)

sys.path.append('drive/MyDrive')
import modelArchs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [ ]:
print(torch.__version__)
print(torchvision.__version__)

1.13.0+cu117
0.14.0+cu117


In [ ]:
train_dataset = pd.read_csv('drive/MyDrive/app_metadata.csv')

diagnostics = sorted(train_dataset.diagnostic.unique().tolist())

def create_label_num(x):
  return diagnostics.index(x)

train_dataset['diagnostic'] = train_dataset['diagnostic'].apply(create_label_num)
train_dataset.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,3,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,1,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,0,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,0,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,1,True,True,False,False,True,True,PAT_684_1302_588.png,True


Κατεβάζω τα δεδομένα.

In [ ]:
urllib.request.urlretrieve("https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip", "imgs.zip")

with zipfile.ZipFile('imgs.zip', 'r') as zip_ref:
  zip_ref.extractall('imgs_folder')

print("Done with unzipping...")

for i in ['imgs_folder/images/imgs_part_1.zip', 'imgs_folder/images/imgs_part_2.zip', 'imgs_folder/images/imgs_part_3.zip']:
  with zipfile.ZipFile(i, 'r') as zip_ref:
    zip_ref.extractall()

os.makedirs('images')

dest_folder = 'images'
for src_folder in ['imgs_part_1', 'imgs_part_2', 'imgs_part_3']:
  file_list = os.listdir(src_folder)
  print("Found %d files in folder: %s" %(len(file_list),src_folder))
  for filename in file_list:
    src_path = os.path.join(src_folder,filename)
    shutil.copy(src_path, dest_folder)

!ls images | wc -l

Done with unzipping...
Found 911 files in folder: imgs_part_1
Found 659 files in folder: imgs_part_2
Found 728 files in folder: imgs_part_3
2298


Αποθηκεύω κάθε εικόνα στον αντίστοιχο φάκελο.

In [ ]:
diagnostics_list = ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

def store_to_corresponding_folder(src_path_prefix,dest_path_prefix,current_dataset):

  #create folder for each dataset
  if os.path.exists(dest_path_prefix):
    print("Folder %s already exists..." %dest_path_prefix)
    return
  os.makedirs(dest_path_prefix)

  #create subfolder for each category...
  for i in diagnostics_list:
    folder_path = os.path.join(dest_path_prefix,i)
    os.makedirs(folder_path)

  #store each image to corresponding folder
  for index, row in current_dataset.iterrows():
    category_label = diagnostics_list[row['diagnostic']]
    img_name = row['img_id']

    src_path = os.path.join(src_path_prefix,img_name)
    dest_path = os.path.join(dest_path_prefix,category_label)
    shutil.copy(src_path,dest_path)

  #print number of elements in each folder
  print("For " + dest_path_prefix + ': ')
  for i in diagnostics_list:
    category_folder = os.path.join(dest_path_prefix,i)
    files_no = len(glob.glob(category_folder + '/*.png'))
    print(i + ': ' + str(files_no))

store_to_corresponding_folder('images','train_imgs', train_dataset)

For train_imgs: 
ACK: 730
BCC: 845
MEL: 52
NEV: 244
SCC: 192
SEK: 235


Υλοποιώ classifier όπου γίνεται επεξεργασία των features από το CNN και των metadata.

In [ ]:
PATH = 'drive/MyDrive/final_app_model.pt'

pretrained_model = modelArchs.get_MobileNetSa(6,False)
pretrained_model.load_state_dict(torch.load(PATH)['model'])

metaModel = modelArchs.get_metaModel_MobileNets(pretrained_model,27,6, metaModelType = 1)
metaModel.to(device)

Λαμβάνω υπόψιν μόνο τα συγκεκριμένα metadata καθώς, ...

In [ ]:
#take into consideration this columns:
metadata_cols = ['img_id', 'age', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation', 'diagnostic']
features_list = ['age', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation']

train_dataset = train_dataset[metadata_cols]
print("Null values in train set: ")
print(train_dataset.isna().sum())

Αντικαθιστώ τα NaN values με UNK.

In [ ]:
train_dataset.fillna('UNK', inplace = True)

Αλλάζω τις categorical variables σε one hot vectors.

In [ ]:
categorical_vars = ['region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation']

def frequency_imputation(train_dataset, column):

  value_counts = train_dataset[column].value_counts().to_dict()

  if 'UNK' in value_counts:
    del value_counts['UNK']

  sum = 0
  for i in value_counts.items():
    sum += i[1]

  for i in value_counts.items():
    value_counts[i[0]] = i[1]/sum

  replacement_choices = list(value_counts.keys())
  replacement_probabilities = list(value_counts.values())

  for lesion in train_dataset.lesion_id.unique():
    random_value = random.choices(replacement_choices, replacement_probabilities)[0]
    train_dataset.loc[(train_dataset['lesion_id'] == lesion) & (train_dataset[column] == 'UNK'), column] = random_value

def one_hot_encode(train_dataset,column):

  unique_vals = sorted(train_dataset[column].unique().tolist())

  train_encodings = []
  for value in train_dataset[column]:

    item_encoding = [0] * len(unique_vals)
    item_encoding[unique_vals.index(value)] = 1
    train_encodings.append(item_encoding)

  train_dataset[column] = train_encodings

for i in categorical_vars:#the categorical vars
  frequency_imputation(train_dataset, i)
  one_hot_encode(train_dataset, i)

train_dataset.head()

Στη συνέχεια, επεξεργαζόμαστε τη μοναδική numerical variable: age.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(np.array(train_dataset['age']).reshape(-1,1))

train_dataset['age'] = scaler.transform(np.array(train_dataset['age']).reshape(-1,1)).tolist()

train_dataset.head()

Τεστ για τη σειρά των features.

In [ ]:
features_cols = train_dataset.iloc[0][features_list].to_numpy().flatten()
features = []
for i in features_cols:
  for j in i:
      features.append(j)
features

[-3.301454057347548,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0]

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.7, 1.3], contrast = [0.7, 1.3], saturation = [0.7, 1.3], hue = [-0.1,0.1]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 20
batch_len = 32

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(metaModel.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

Υλοποιώ το δικό μου dataset, καθώς θα θέλαμε, εκτός από την εικόνα, να γίνονται load και τα αντίστοιχα metadata.

In [ ]:
class ImageFolderWithFileNames(datasets.ImageFolder):

    def __init__(self, root_dir, train_transformations, current_dataset, features_list):

      super().__init__(root_dir,train_transformations)

      self.current_dataset = current_dataset
      self.features_list = features_list

    def __getitem__(self, index):

      path, target = self.imgs[index]

      filename = path[path.rfind('/') + 1:]
      metadata_features = self.get_metadata_features(filename)

      img = self.loader(path)
      if self.transform is not None:
        img = self.transform(img)
      if self.target_transform is not None:
        target = self.target_transform(target)

      return metadata_features,img,target

    def get_metadata_features(self,filename):

      features = []

      index_row = self.current_dataset.index[self.current_dataset['img_id'] == filename].tolist()[0]#get index of the row that corresponds to the image
      feature_cols = self.current_dataset.iloc[index_row][self.features_list].to_numpy().flatten()

      for i in feature_cols:
        for j in i:
          features.append(j)

      return torch.tensor(features)

Ορίζουμε weights για κάθε κλάση, ώστε να αντιμετωπίσουμε το πρόβλημα του imbalanced dataset.

In [ ]:
def get_weights_for_imgs(images, n_categories):

    n_images = len(images)

    imgs_per_category = [0] * n_categories
    for _, image_category in images:
      imgs_per_category[image_category] += 1

    weight_per_category = [0] * n_categories # έχουμε 6 κατηγορίες
    for i in range(n_categories):
      weight_per_category[i] = 1 / float(imgs_per_category[i])#float(n_images) / float(imgs_per_category[i])

    weights = [0] * n_images
    for idx, (_, image_category) in enumerate(images):
        weights[idx] = weight_per_category[image_category]

    return weights

train_dataset_imgs = ImageFolderWithFileNames('train_imgs', transforms_train, train_dataset, features_list)

len_train_dataset_imgs = len(train_dataset_imgs)

print("Length of train_dataset: ", len_train_dataset_imgs)

weights = get_weights_for_imgs(train_dataset_imgs.imgs, len(train_dataset_imgs.classes))
weights = torch.DoubleTensor(weights)
train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader_imgs = torch.utils.data.DataLoader(train_dataset_imgs, batch_size = batch_len, sampler = train_sampler, pin_memory=True)

Length of train_dataset:  2298


Το βασικό train loop.

In [ ]:
for epoch in range(1, epochs + 1):

  metaModel.train()

  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (metadata_features, imgs, labels) in enumerate(train_dataloader_imgs,1):

      metadata_features = metadata_features.to(device)
      imgs = imgs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = metaModel(metadata_features,imgs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

try:
  metaModel.eval()
  checkpoint = {'model': metaModel.state_dict()}
  checkpoint_fn = 'drive/MyDrive/final_model_metadata.pt'
  torch.save(checkpoint, checkpoint_fn)
except:
  print("Error in writing general checkpoint")